In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def developer(developer):
    df = pd.read_parquet('./Datasets/developer_endpoint.parquet')
    # Filtramos por el desarrollador ingresado.
    df = df[df['developer'] == developer]
    # Creamos la columna año en base a la columna release_date.
    df['Año'] = df['release_date'].dt.year
    # Agrupamos por año y contamos la cantidad de items.
    df_year = df.groupby('Año').size().reset_index(name='Cantidad de Items')
    # Agrupamos por año y sumamos la cantidad de items free.
    df_free = df.groupby('Año')['free'].sum().reset_index(name='free')
    # Hacemos un merge de ambos dataframes mediante la columna Año.
    df = pd.merge(df_year, df_free, on='Año')
    # Calculamos el porcentaje de contenido free.
    df['Contenido Free'] = round(df['free'] / df['Cantidad de Items'] * 100, 2)
    # Convertimos el porcentaje a string y le agregamos el símbolo %.
    df['Contenido Free'] = df['Contenido Free'].apply(lambda x: f'{x}%')
    # Eliminamos la columna free.
    df.drop(columns=['free'], inplace=True)
    # Convertimos el dataframe a un diccionario.
    df = df.to_dict('records')
    # Devolvemos el resultado.
    return df

In [3]:
developer('Valve')

[{'Año': 1998, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'Año': 1999, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'Año': 2000, 'Cantidad de Items': 2, 'Contenido Free': '0.0%'},
 {'Año': 2001, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'Año': 2003, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'Año': 2004, 'Cantidad de Items': 5, 'Contenido Free': '0.0%'},
 {'Año': 2006, 'Cantidad de Items': 2, 'Contenido Free': '0.0%'},
 {'Año': 2007, 'Cantidad de Items': 3, 'Contenido Free': '33.33%'},
 {'Año': 2008, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'Año': 2009, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'Año': 2010, 'Cantidad de Items': 2, 'Contenido Free': '50.0%'},
 {'Año': 2011, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'Año': 2012, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'Año': 2016, 'Cantidad de Items': 1, 'Contenido Free': '100.0%'},
 {'Año': 2017, 'Cantidad de Items': 2, 'Contenido Free': '0.0%'}]

In [33]:
df = pd.read_parquet('./Datasets/game_recommendation.parquet')
df

,item_id,title,features
0,442050,Just Cause™ 3 DLC: Kousavá Rifle,"Action, Adventure"
1,339386,X-Plane 10 AddOn - Carenado - PA46 Malibu Mira...,Simulation
2,436160,The Panic Room,"Free to Play, Hidden Object"
3,710040,Fantasy Grounds - Feats of Legend: 20 Story Fe...,"Indie, RPG, Strategy"
4,506971,DARK SOULS™ III - The Ringed City™,"Action, Action RPG, Adventure, Atmospheric, Ch..."
...,...,...,...
9995,538070,Bad Dream: Coma,"Adventure, Atmospheric, Gore, Hand-drawn, Horr..."
9996,468733,Tabletop Simulator - Viticulture,"Casual, Indie, RPG, Simulation, Strategy"
9997,715290,SEGFAULT,"Action, Adventure, Indie"
9998,436140,Empire of the Gods,"Card Game, Casual, Indie, Strategy"


In [34]:
def game_recommendation(item_id):
    item_id = str(item_id)
    df = pd.read_parquet('./Datasets/game_recommendation.parquet')
    if item_id not in df['item_id'].values:
        return 'ID no encontrado'
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['features'])
    cosine_sim = cosine_similarity(tfidf_matrix)
    idx = df[df['item_id'] == item_id].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    recommended_games = df['title'].iloc[[i[0] for i in sim_scores]].tolist()
    return {"recommended_games": recommended_games}

In [35]:
game_recommendation(240)

{'recommended_games': ['Umbrella Corps™/Biohazard Umbrella Corps™',
  'Ravenfield',
  'Verdun',
  'Red Orchestra 2: Heroes of Stalingrad with Rising Storm',
  "Tom Clancy's Ghost Recon® Wildlands"]}